# AutoRAG - DataCreation Pipeline

# Module

In [1]:
import warnings
warnings.filterwarnings(action='ignore')
import os
import sys
import dotenv
import nest_asyncio
dotenv.load_dotenv()
# nest_asyncio 적용
nest_asyncio.apply()

import pandas as pd

# AutoRAG
from autorag.parser import Parser
from autorag.chunker import Chunker
from autorag.data.qa.schema import Raw, Corpus
from autorag.data.qa.filter.dontknow import dontknow_filter_rule_based
from autorag.data.qa.generation_gt.llama_index_gen_gt import make_basic_gen_gt,make_concise_gen_gt
from autorag.data.qa.query.llama_gen_query import factoid_query_gen
from autorag.data.qa.sample import random_single_hop

# llamaindex
from llama_index.llms.openai import OpenAI

# RawData Parse

In [16]:
# parser 정의
parser = Parser(data_path_glob="/home/livin/rag_pipeline/AutoRAG/data/*.pdf")

In [15]:
parser.start_parsing("/home/livin/rag_pipeline/AutoRAG/configs/data_creation_settings.yaml")

[10/11/24 15:11:20] INFO     [parser.py:33] >> Parsing Start...                                        ]8;id=518716;file:///home/livin/anaconda3/envs/rag_pipeline/lib/python3.11/site-packages/autorag/parser.py\parser.py]8;;\:]8;id=594094;file:///home/livin/anaconda3/envs/rag_pipeline/lib/python3.11/site-packages/autorag/parser.py#33\33]8;;\

TypeError: chunker_node.<locals>.wrapper() missing 1 required positional argument: 'parsed_result'

In [17]:
# Parse 결과 확인
df = pd.read_parquet("../result/parse/hyundai_upstage/0.parquet")
df

# Chunking

In [23]:
chunker = Chunker.from_parquet(parsed_data_path="../result/parse/hyundai_upstage/0.parquet", project_dir="../result/chunk")
chunker.start_chunking("../settings.yaml")

[10/07/24 10:24:53] INFO     [chunker.py:48] >> Chunking Start...                                     ]8;id=55257;file:///home/livin/anaconda3/envs/rag_pipeline/lib/python3.11/site-packages/autorag/chunker.py\chunker.py]8;;\:]8;id=564516;file:///home/livin/anaconda3/envs/rag_pipeline/lib/python3.11/site-packages/autorag/chunker.py#48\48]8;;\

                    INFO     [base.py:22] >> Running chunker - llama_index_chunk module...               ]8;id=182080;file:///home/livin/anaconda3/envs/rag_pipeline/lib/python3.11/site-packages/autorag/data/chunk/base.py\base.py]8;;\:]8;id=112054;file:///home/livin/anaconda3/envs/rag_pipeline/lib/python3.11/site-packages/autorag/data/chunk/base.py#22\22]8;;\

[10/07/24 10:24:56] INFO     [chunker.py:55] >> Chunking Done!                                        ]8;id=540228;file:///home/livin/anaconda3/envs/rag_pipeline/lib/python3.11/site-packages/autorag/chunker.py\chunker.py]8;;\:]8;id=263503;file:///home/livin/anaconda3/envs/rag_pipeline/lib/python3.11/site-packages/autorag/chunker.py#55\55]8;;\

In [25]:
# Chunk 결과 확인
corpus_df = pd.read_parquet("../result/chunk/hyundai_upstage/0.parquet")
corpus_df

,doc_id,contents,path,start_end_idx,metadata
0,af0d9d7f-a1ca-40a9-9c16-67676f0bc620,파일 제목: hyndai.pdf\n 내용: <h1 id='0' style='font...,/home/livin/rag_pipeline/AutoRAG/data/hyndai.pdf,"[0, 1841]","{'last_modified_datetime': '2024-09-06', 'next..."
1,d982d5c6-41fb-4d96-ad9a-76929fe018b5,파일 제목: hyndai.pdf\n 내용: 포 함 하 여 설 명<br>하 고 있 습...,/home/livin/rag_pipeline/AutoRAG/data/hyndai.pdf,"[1458, 3172]","{'last_modified_datetime': '2024-09-06', 'next..."
2,ba009f64-4655-4b0c-b958-684390d73789,파일 제목: hyndai.pdf\n 내용: . 한 국 교 통 안 전 공 단 자 동 ...,/home/livin/rag_pipeline/AutoRAG/data/hyndai.pdf,"[2909, 4374]","{'last_modified_datetime': '2024-09-06', 'next..."
3,f5a378aa-92f4-4be1-a972-4871077582ed,파일 제목: hyndai.pdf\n 내용: 러 스 터</td><td>4</td></...,/home/livin/rag_pipeline/AutoRAG/data/hyndai.pdf,"[4067, 7508]","{'last_modified_datetime': '2024-09-06', 'next..."
4,0a474ba9-d0d5-45d7-a713-dc6e6e367014,파일 제목: hyndai.pdf\n 내용: 이용한 비상 제동이 필요한 경우 .......,/home/livin/rag_pipeline/AutoRAG/data/hyndai.pdf,"[7117, 8718]","{'last_modified_datetime': '2024-09-06', 'next..."
...,...,...,...,...,...
834,1ee41678-458e-4f7b-a148-ac3a925ddd11,파일 제목: hyndai.pdf\n 내용: 차량으로 견인 시 8-20<br>선루프 ...,/home/livin/rag_pipeline/AutoRAG/data/hyndai.pdf,"[1105764, 1106878]","{'last_modified_datetime': '2024-09-06', 'next..."
835,49cf11d4-a9c3-4cd9-b775-24a117bd4702,파일 제목: hyndai.pdf\n 내용: 및 용량 1-14<br>교차로/건널목에서...,/home/livin/rag_pipeline/AutoRAG/data/hyndai.pdf,"[1106656, 1107960]","{'last_modified_datetime': '2024-09-06', 'next..."
836,9cdec7bf-44f4-46c9-b67f-21d987d2087d,파일 제목: hyndai.pdf\n 내용: 고전압 모터 커텍터(엔진룸) 0-18<b...,/home/livin/rag_pipeline/AutoRAG/data/hyndai.pdf,"[1107759, 1108852]","{'last_modified_datetime': '2024-09-06', 'next..."
837,ed053d51-2334-45a7-b738-9c80d2403657,파일 제목: hyndai.pdf\n 내용: 하이빔 보조 작동 5-95<br>드라이브...,/home/livin/rag_pipeline/AutoRAG/data/hyndai.pdf,"[1108613, 1109837]","{'last_modified_datetime': '2024-09-06', 'next..."


# QA Creation

In [2]:
# load RAW data
raw_df = pd.read_parquet("../results/chunk/nursing/2_nursing_chunk/0.parquet")
raw_instance = Raw(raw_df)

# load Corpus data
corpus_df = pd.read_parquet("../results/chunk/nursing/2_nursing_chunk/0.parquet")
corpus_instance = Corpus(corpus_df, raw_instance)

In [5]:
# llm 선언
llm = OpenAI(model="gpt-4o-mini")

# QA 데이터 생성 파이프라인
initial_qa = (
    corpus_instance.sample(random_single_hop, random_state=9999, n=20) # 몇개의 corpus를 뽑을지 결정
    .map(
        lambda df: df.reset_index(drop=True), # Drop index
    )
    .make_retrieval_gt_contents() # retrieval_gt_contents 생성
    .batch_apply(
        factoid_query_gen,  # query generation
        llm=llm,
        lang="ko",
    )
    .batch_apply(
        make_basic_gen_gt,  # answer generation (basic)
        llm=llm,
        lang="ko",
    )
    .filter(
        dontknow_filter_rule_based,  # unanswerable filter
        lang="ko",
    )
)


# 생성된 qa 저장
initial_qa.to_parquet('../results/qa/nursing/qa_6.parquet', '../results/qa/nursing/corpus.parquet')

[10/16/24 13:59:42] INFO     [_client.py:1786] >> HTTP Request: POST                                ]8;id=543288;file:///home/livin/anaconda3/envs/rag_pipeline/lib/python3.11/site-packages/httpx/_client.py\_client.py]8;;\:]8;id=85629;file:///home/livin/anaconda3/envs/rag_pipeline/lib/python3.11/site-packages/httpx/_client.py#1786\1786]8;;\
                             https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"                          

                    INFO     [_client.py:1786] >> HTTP Request: POST                                ]8;id=682331;file:///home/livin/anaconda3/envs/rag_pipeline/lib/python3.11/site-packages/httpx/_client.py\_client.py]8;;\:]8;id=402999;file:///home/livin/anaconda3/envs/rag_pipeline/lib/python3.11/site-packages/httpx/_client.py#1786\1786]8;;\
                             https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"                          

                    INFO     [_client.py:1786] >> HTTP Request: POST                                ]8;id=660565;file:///home/livin/anaconda3/envs/rag_pipeline/lib/python3.11/site-packages/httpx/_client.py\_client.py]8;;\:]8;id=520306;file:///home/livin/anaconda3/envs/rag_pipeline/lib/python3.11/site-packages/httpx/_client.py#1786\1786]8;;\
                             https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"                          

                    INFO     [_client.py:1786] >> HTTP Request: POST                                ]8;id=577176;file:///home/livin/anaconda3/envs/rag_pipeline/lib/python3.11/site-packages/httpx/_client.py\_client.py]8;;\:]8;id=664436;file:///home/livin/anaconda3/envs/rag_pipeline/lib/python3.11/site-packages/httpx/_client.py#1786\1786]8;;\
                             https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"                          

                    INFO     [_client.py:1786] >> HTTP Request: POST                                ]8;id=308328;file:///home/livin/anaconda3/envs/rag_pipeline/lib/python3.11/site-packages/httpx/_client.py\_client.py]8;;\:]8;id=601770;file:///home/livin/anaconda3/envs/rag_pipeline/lib/python3.11/site-packages/httpx/_client.py#1786\1786]8;;\
                             https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"                          

...

                    INFO     [_client.py:1786] >> HTTP Request: POST                                ]8;id=443601;file:///home/livin/anaconda3/envs/rag_pipeline/lib/python3.11/site-packages/httpx/_client.py\_client.py]8;;\:]8;id=46368;file:///home/livin/anaconda3/envs/rag_pipeline/lib/python3.11/site-packages/httpx/_client.py#1786\1786]8;;\
                             https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"                          

[10/16/24 13:59:50] INFO     [_client.py:1786] >> HTTP Request: POST                                ]8;id=763858;file:///home/livin/anaconda3/envs/rag_pipeline/lib/python3.11/site-packages/httpx/_client.py\_client.py]8;;\:]8;id=891822;file:///home/livin/anaconda3/envs/rag_pipeline/lib/python3.11/site-packages/httpx/_client.py#1786\1786]8;;\
                             https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"                          

[10/16/24 13:59:52] INFO     [_client.py:1786] >> HTTP Request: POST                                ]8;id=976250;file:///home/livin/anaconda3/envs/rag_pipeline/lib/python3.11/site-packages/httpx/_client.py\_client.py]8;;\:]8;id=601667;file:///home/livin/anaconda3/envs/rag_pipeline/lib/python3.11/site-packages/httpx/_client.py#1786\1786]8;;\
                             https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"                          

[10/16/24 13:59:54] INFO     [_client.py:1786] >> HTTP Request: POST                                ]8;id=773972;file:///home/livin/anaconda3/envs/rag_pipeline/lib/python3.11/site-packages/httpx/_client.py\_client.py]8;;\:]8;id=618515;file:///home/livin/anaconda3/envs/rag_pipeline/lib/python3.11/site-packages/httpx/_client.py#1786\1786]8;;\
                             https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"                          

[10/16/24 13:59:59] INFO     [_client.py:1786] >> HTTP Request: POST                                ]8;id=96179;file:///home/livin/anaconda3/envs/rag_pipeline/lib/python3.11/site-packages/httpx/_client.py\_client.py]8;;\:]8;id=77878;file:///home/livin/anaconda3/envs/rag_pipeline/lib/python3.11/site-packages/httpx/_client.py#1786\1786]8;;\
                             https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"                          

In [6]:
# 생성된 qa 확인
initial_qa.data.head(2)

,qid,retrieval_gt,retrieval_gt_contents,query,generation_gt
0,8cc05e9e-b7aa-4ebf-a824-263380d9950e,[[3ea0aa60-7a9c-45ef-8538-cd9bff16b9b3]],[[파일 제목: 15장 심장장애 대상자 간호-(2)\n...,"경피적 인공심박동기(Transcutaneous pacing, TCP)는 어떤 방법으...","[경피적 인공심박동기(Transcutaneous pacing, TCP)는 피부에 부..."
1,07090ade-47ae-4bcf-a26c-b200f250d4a1,[[f72ed50c-8b79-4a08-b744-68cf4c39e68e]],[[파일 제목: 21장 근골격계 장애\n 내용: 간호</p><...,고유의 구조와 기능을 가진 인간의 뼈는 총 몇 개입니까?,[인간의 뼈는 총 206개입니다.]


In [16]:
# Chunk 결과 확인
corpus_df = pd.read_parquet("/home/livin/rag_pipeline/AutoRAG/results/qa/hyundai_upstage_qa_3.parquet")
corpus_df.head(2)

,qid,query,retrieval_gt,generation_gt
0,052b323b-fa7f-41f0-9dd6-41791d2a5d6f,에어컨을 사용한 후 차량 밑바닥에서 물이 떨어지는 이유는 무엇인가요?,[[5d54c7a1-134d-4c29-a07f-68f68d7979a8]],[에어컨을 사용한 후 차량 밑바닥에서 물이 떨어지는 것은 정상적인 현상입니다.]
1,3e372364-e950-41bb-beff-dad5cecfdedd,"USB 충전 단자는 어떤 용도로 사용되며, 이 단자를 통해 미디어를 재생할 수 있습니까?",[[596ccf0f-1e57-41dc-908a-a3f99803ba80]],"[USB 충전 단자는 충전 전용으로 사용되며, 이 단자를 통해서는 인포테인먼트 시스..."


# QA Integration
* 다양한 QA 셋을 통합하여 하나의 QA 셋으로 만듦

In [7]:
qa_list = ["../results/qa/nursing/qa_1.parquet", "../results/qa/nursing/qa_2.parquet","../results/qa/nursing/qa_3.parquet","../results/qa/nursing/qa_4.parquet","../results/qa/nursing/qa_5.parquet","../results/qa/nursing/qa_6.parquet"]

In [8]:
# 여러 qa 파일을 하나의 qa 파일로 합치기
qa_df = pd.concat([pd.read_parquet(qa) for qa in qa_list], ignore_index=True)

# qa 저장
qa_df.to_parquet("../results/qa/nursing/nursing_qa.parquet")

# Test

In [20]:
qa_df = pd.read_parquet('../results/qa/hyundai_upstage_qa_1234.parquet')
corpus_df = pd.read_parquet('../results/qa/hyundai_upstage_corpus.parquet')

In [21]:
qa_df.head(2)

,qid,query,retrieval_gt,generation_gt
0,f81a5a16-1c6b-4e59-af51-6e4b0959b456,고속도로 주행 보조 설정은 몇 페이지에 나와 있습니까?,[[937e0eea-65b2-4195-b374-a72c6eccb872]],[고속도로 주행 보조 설정은 7-81 페이지에 나와 있습니다.]
1,894f74ee-f5e1-41d8-a5ef-484b50f28bb0,디지털 센터 미러의 올바른 사용에 대한 경고 및 지침을 따르지 않으면 어떤 위험이 ...,[[05115da3-694b-49c1-9004-c59462259054]],[디지털 센터 미러의 올바른 사용에 대한 경고 및 지침을 따르지 않으면 심각한 사고...


In [23]:
len(qa_df)

120

In [24]:
corpus_df.head(2)

,doc_id,contents,metadata
0,af0d9d7f-a1ca-40a9-9c16-67676f0bc620,파일 제목: hyndai.pdf\n 내용: <h1 id='0' style='font...,"{'last_modified_datetime': '2024-09-06', 'next..."
1,d982d5c6-41fb-4d96-ad9a-76929fe018b5,파일 제목: hyndai.pdf\n 내용: 포 함 하 여 설 명<br>하 고 있 습...,"{'last_modified_datetime': '2024-09-06', 'next..."


In [25]:
idx = 0
print(qa_df.loc[idx, 'query'])
print(qa_df.loc[idx, 'generation_gt'])

고속도로 주행 보조 설정은 몇 페이지에 나와 있습니까?
['고속도로 주행 보조 설정은 7-81 페이지에 나와 있습니다.']


In [26]:
corpus_df[corpus_df["doc_id"] == qa_df.loc[idx, "retrieval_gt"][0][0]]["contents"].item()

"파일 제목: hyndai.pdf\n 내용: 해제 기능 5-30<br>고속도로 주행 보조 설정 7-81<br>전자식 차일드 락 5-30<br>고속도로 주행 보조 이상 및 제한 사항 7-88<br>차 밖에서의 잠금/잠금 해제 5-26<br>고속도로 주행 보조 작동 7-82<br>차 안에서의 잠금/잠금 해제 5-28<br>공조 장치용 에어필터 9-20<br>차속 감응 자동 도어 잠금 장치 5-29<br>에어필터 교체 9-20<br>충돌 감지 자동 도어 잠금 해제 장치 5-29<br>에어필터 점검 9-20<br>도어/테일게이트/<br>교차로나 건널목에서의 긴급 상황 대처 2-9<br>유리창을 열고 닫을 때 조심 2-8<br>규격 타이어 장착 및 타이어 공기압 수시 점검<br>듀얼 와이드 선루프 5-59<br>2-4<br>물체 끼임 인식 기능 5-61<br>기타 주의 사항 2-11<br>선루프 열림 경고 5-62<br>선루프 초기화 5-62<br>나 선블라인드 (앞)/파워 선블라인드 (뒤) 5-59<br>슬라이드 열림/닫힘 (앞좌석) 5-60<br>내 비게이션 기반 스마트 크루즈 컨트롤 (NSCC)<br>틸트 열림/닫힘 (앞좌석) 5-60<br>7-71<br>내비게이션 기반 스마트 크루즈 컨트롤 설정<br>7-72 마<br>내비게이션 기반 스마트 크루즈 컨트롤 작동<br>7-72 무 선 소프트웨어 업데이트 5-85<br>작동 제한 사항 7-74 소프트웨어 업데이트 승인 5-85<br>냉각수 점검 9-15 소프트웨어 자동 다운로드 5-85<br>냉각수량 점검 및 보충 9-15 미러 5-42<br>냉각수의 교체 9-17 실내 미러 5-42<br>실외 미러 5-48</p><footer id='14' style='font-size:18px'>I-2</footer><br><h1 id='15' style='font-size:14px'>MX5HEV_ko_KR.book Page 3</h1><p id='16' data-category='paragraph' style='font-siz